In [1]:
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf
import os
import pickle

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

from music21 import converter, instrument, stream, note, chord

#Run version 2.1.6
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

### Improved LSTM with attention

In [2]:
##### PARAMETERS #####
seq_length = 100
#num_files_used

NameError: name 'num_files_used' is not defined

In [3]:
def get_notes():
    """ Get all the notes and chords from the midi files - Call BEFORE train """
    notes = []
    
    print("Parsing files...")
    
    for file in glob.glob("../raw_data/midi_chopin_format_0_dataset/*.mid")[:10]:
        stream_file = converter.parse(file)

        #print("Parsing %s" % file)
                
        components_to_parse = []
        for element in stream_file.recurse():
            components_to_parse.append(element)
        
        #components_to_parse = stream_file.flat.notes #return to this to preview model output 10 epochs 1st training
    
        for element in components_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(float(element.quarterLength)))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(float(element.quarterLength)))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(float(element.quarterLength)))

    with open('../raw_data/notes/notes.pickle', 'wb') as f:
        pickle.dump(notes, f)
    
    return notes

In [4]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = seq_length


    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = utils.to_categorical(network_output)

    return (network_input, network_output)


In [5]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [6]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = os.path.abspath("model_checkpoints_weights/weights-1LSTMAtt1LSTMLayer-{epoch:03d}-{loss:.4f}.hdf5")
    checkpoint = ModelCheckpoint(
        filepath,
        period=3, #Every 10 epochs
        monitor='loss',
        verbose=1,
        save_best_only=False,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)


In [7]:
def train_network(notes, n_vocab):
    """ Train a Neural Network to generate music """
    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [546]:
#load files in
notes = get_notes()

# get amount of pitch names
n_vocab = len(set(notes))
n_vocab

Parsing files...


2018

In [559]:
notes

['1.5 0.5',
 'G#3 0.5',
 '1.5.8 2.0',
 'B-4 1.0',
 'rest 0.75',
 'C#5 0.25',
 'rest 0.5',
 'G#3 0.5',
 'G#3 0.5',
 'G#3 0.5',
 '1.5 0.5',
 'G#3 0.5',
 '0.3.6 3.0',
 'C#5 1.0',
 'rest 0.5',
 'G#3 0.5',
 'G#3 0.5',
 'G#3 0.5',
 'B-3 0.5',
 'G#3 0.5',
 '1.5 0.5',
 'G#3 0.5',
 '3.6 0.75',
 '5.8 0.25',
 '6.10 2.0',
 '8.1 0.5',
 'G#3 0.5',
 'C4 3.0',
 'F5 1.0',
 'rest 0.5',
 'G#3 1.0',
 'G#3 0.5',
 'G#3 0.5',
 'G#3 1.0',
 'rest 0.5',
 'G#4 0.5',
 'G#3 0.5',
 'F#4 0.5',
 '3.8 0.5',
 'C#5 0.75',
 '3 0.25',
 '0.5.6 0.25',
 '2.3 0.25',
 '5.8 0.25',
 'F#5 0.25',
 'F5 1.5',
 'rest 0.5',
 '1.5 0.5',
 'G#3 0.5',
 'rest 0.5',
 '3.5 0.25',
 'rest 0.25',
 'C4 2.0',
 'rest 2.0',
 '1.5 0.5',
 'G#3 0.5',
 'G#4 2.0',
 'B-4 1.0',
 'rest 0.75',
 'C#5 0.25',
 '1.5 3.0',
 'rest 1.5',
 'G#3 0.5',
 'G#3 0.5',
 'G#3 0.5',
 'G#3 0.5',
 'G#3 0.5',
 '0.3.6 3.0',
 'C#5 1.0',
 'rest 0.5',
 'G#3 0.5',
 'G#3 0.5',
 'G#3 0.5',
 'B-3 0.5',
 'G#3 0.5',
 '1.5 0.5',
 'G#3 0.5',
 '3.6 0.75',
 '5.8 0.25',
 '6.10 2.0',
 '8.1 0.

In [547]:
network_input, network_output = prepare_sequences(notes, n_vocab)

In [548]:
network_input.shape

(15818, 10, 1)

In [549]:
network_input[2].shape

(10, 1)

In [550]:
#train
train_network(notes, n_vocab) 

Epoch 1/10


2022-12-09 17:23:20.494421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:23:21.139328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:23:21.166340: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:23:21.575647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:23:21.736471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:23:22.059746: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:23:22.098454: I tensorflow/core/grappler/optimizers/cust

248/248 [==============================] - 17s 47ms/step - loss: 6.0897
Epoch 2/10
248/248 [==============================] - 10s 40ms/step - loss: 5.8017
Epoch 3/10
248/248 [==============================] - ETA: 0s - loss: 5.7202
Epoch 3: saving model to /Users/aurelie_emma/code/aeraymd/chopin_28_prelude/notebooks-aurelie/model_checkpoints_weights/weights-1LSTMAtt1LSTMLayer-003-5.7202.hdf5
248/248 [==============================] - 10s 41ms/step - loss: 5.7202
Epoch 4/10
248/248 [==============================] - 10s 41ms/step - loss: 5.6303
Epoch 5/10
248/248 [==============================] - 10s 41ms/step - loss: 5.5902
Epoch 6/10
247/248 [============================>.] - ETA: 0s - loss: 5.5655
Epoch 6: saving model to /Users/aurelie_emma/code/aeraymd/chopin_28_prelude/notebooks-aurelie/model_checkpoints_weights/weights-1LSTMAtt1LSTMLayer-006-5.5659.hdf5
248/248 [==============================] - 10s 41ms/step - loss: 5.5659
Epoch 7/10
248/248 [==============================] - 1

In [551]:
weights_path = 'model_checkpoints_weights/weights-1LSTMAtt1LSTMLayer-009-5.4853_latest.hdf5'

In [552]:
def create_network_add_weights(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    
    # Load the weights to each node
    #model.build(input_shape=network_input.shape)
    #model.load_weights(weights_path)
    
    return model

In [563]:
def prepare_sequences_output(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = seq_length
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)


In [554]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [555]:
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [560]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif('rest' in pattern):
            #new_rest = note.Rest(pattern)
            new_rest = note.Rest()
            new_rest.offset = offset
            new_rest.storedInstrument = instrument.Piano() #???
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += convert_to_float(duration)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='../raw_data/model_outputs/test_output_with_rest.mid')
    
    #midi_stream.show('midi')


In [561]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    notes = get_notes()
    #with open('../raw_data/notes/notes.pickle', 'rb') as f:
        #notes = pickle.load(f)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_output(notes, pitchnames, n_vocab)
    model = create_network_add_weights(normalized_input, n_vocab)
    print(normalized_input.shape[1], normalized_input.shape[2])
    model.build(input_shape=(None, normalized_input.shape[1], normalized_input.shape[2])) #(None,100, 1))
    model.load_weights(weights_path)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)


In [562]:
#RUN THE GENERATOR

generate()


Parsing files...
10 1


/Users/aurelie_emma/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2022-12-09 17:36:23.500743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:36:23.630688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:36:23.643648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 17:36:23.734635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registr

In [518]:

#Test stuff

#with open('raw_data/notes', 'rb') as filepath:
    #notes = pickle.load(filepath)

# Get all pitch names
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))
    
#print(pitchnames)
print(n_vocab)
#print(notes)
print(len(notes))


7541
144400
